In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

In [51]:
!pip install imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 2.6 MB/s eta 0:00:0000:0100:01


---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [68]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_df = pd.read_csv("Resources/lending_data.csv")

# Review the DataFrame
# lending_df.head()
# lending_df.dtypes
# lending_df.describe()
lending_df.columns

Index(['loan_size', 'interest_rate', 'borrower_income', 'debt_to_income',
       'num_of_accounts', 'derogatory_marks', 'total_debt', 'loan_status'],
      dtype='object')

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [14]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df["loan_status"].values.reshape(-1,1)

# Separate the X variable, the features
X = lending_df.copy()
X.drop("loan_status",axis=1, inplace=True)

In [18]:
# Review the y variable Series
y[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [15]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [23]:
# Check the balance of our target values
lending_df["loan_status"].value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [28]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
y_train.shape

(58152, 1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [31]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr_model = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
lr_model.fit(X_train, y_train)

/Users/oaklandsveryown/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [32]:
# Make a prediction using the testing data
lr_testing_predictions = lr_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [46]:
# Print the balanced_accuracy score of the model
print(f"The model's balanced accuracy score is: {balanced_accuracy_score(y_test, lr_testing_predictions)}")

The model's balanced accuracy score is: 0.9442676901753825


In [47]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, lr_testing_predictions)

array([[18679,    80],
       [   67,   558]])

In [49]:
# Print the classification report for the model
target_names = ["Healthy Loan", "High-Risk Loan"]
print(classification_report(y_test,lr_testing_predictions,target_names=target_names))

                precision    recall  f1-score   support

  Healthy Loan       1.00      1.00      1.00     18759
High-Risk Loan       0.87      0.89      0.88       625

      accuracy                           0.99     19384
     macro avg       0.94      0.94      0.94     19384
  weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** 

The model has a very high overall accuracy score, but because of the imbalance in our target values, we have to look deeper than that to understand its effectiveness. Our target value has many more Healthy Loan values than High-Risk Loan values, so we expect the model to be able to identify Healthy Loans, and we can see that in the precision and recall of 100%. 

For High-Risk Loans, there are many fewer for the model to look at and the precision and recall are much lower (87% and 89%). So while the overall accuracy score is very high, when we drill down into the model's ability to correctly identify High-Risk Loans, we are  below the performance threshold we would like to be at.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [53]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [58]:
# Count the distinct values of the resampled labels data
X_resampled.shape
y_resampled.shape

(112554,)

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [61]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr_model2 = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using the resampled training data
lr_model2.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
resampled_lr_predictions = lr_model2.predict(X_test)


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [62]:
# Print the balanced_accuracy score of the model 
print(f"This model's balanced accuracy score is: {balanced_accuracy_score(y_test, resampled_lr_predictions)}")

This model's balanced accuracy score is: 0.9959744975744975


In [63]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, resampled_lr_predictions)

array([[18668,    91],
       [    2,   623]])

In [65]:
# Print the classification report for the model
target_names = ["Healthy Loan", "High-Risk Loan"]
print(classification_report(y_test,resampled_lr_predictions,target_names=target_names))

                precision    recall  f1-score   support

  Healthy Loan       1.00      1.00      1.00     18759
High-Risk Loan       0.87      1.00      0.93       625

      accuracy                           1.00     19384
     macro avg       0.94      1.00      0.96     19384
  weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** 

Now that we have oversampled the data used to train the model, we see that it performs much better at positive identification of High-Risk Loans: the recall score is 1. Our precision score for High-Risk Loans is still only 87%, so the model is producing some false positive results, but we see no room for false negatives in this recall score. All of the High-Risk Loans that are truly high risk have been classified as such. 

